In [65]:
import pandas as pd
import gzip
import json
import re
import datetime
import numpy as np

In [66]:
def get_data_from_date(specific_date):
    return all_data[(all_data["Date"] == end_date)]

In [102]:
class Coinmarketcap(object):
    @staticmethod
    def load_data(source_datapath):
        prices = {}
        all_data = pd.read_csv(source_datapath, sep='\t', header=0, encoding="utf-8", dtype={'Market Cap': np.float64, 'Total Market Cap': np.float64})
        all_data["Market Cap Share"] = all_data["Market Cap"]/all_data["Total Market Cap"]
        for index, value in all_data['Date'].items():
            date = value
            symbol = all_data['Symbol'][index]
            price = all_data['Price'][index]
            if prices.get(date) is None:
                prices[date] = {}
            prices[date][symbol] = price
        return all_data, prices

    def __init__(self, source_datapath):
        self.data, self.prices = self.load_data(source_datapath)
    
    def get_coin_price(self, date, coin_symbol):
        try:
            return float(self.prices[date][coin_symbol])
        except:
            print("ERROR: price not found for coin %s (%s)" % (coin_symbol, date))
            return 0

    def coins_to_usd(self, date, coins):
        amount_usd = 0
        for coin_symbol, amount in coins.items():
            coin_usd = self.get_coin_price(date, symbol)
            amount_usd += amount * coin_usd
        return amount_usd
    
    def buy_coin(self, date, symbol, amount_usd):
        return amount_usd/self.get_coin_price(date, symbol)
    
    def buy_coins(self, date, symbols, amount_usd, strategy="even"):
        symbols_length = len(symbols)
        coins = {}
        if strategy == "market_cap":
            coins_market_cap = {}
            total_market_cap = 0
            for coin_symbol in symbols:
                coins_market_cap[coin_symbol] = self.get_coin_market_cap(date, coin_symbol)
                total_market_cap += coins_market_cap[coin_symbol]
        for coin_symbol in symbols:
            coin_amount_usd = 0
            if strategy == "even":
                coin_amount_usd = (amount_usd/symbols_length)
            elif strategy == "market_cap":
                coin_amount_usd = (amount_usd*(coins_market_cap[coin_symbol]/total_market_cap))
            coins[coin_symbol] = self.buy_coin(date, coin_symbol, coin_amount_usd)
        return coins
    
    def sell_coins(self, date, coins):
        amount_usd = 0
        for coin_symbol, coin_amount in coins.items():
            coin_price = self.get_coin_price(date, coin_symbol)
            amount_usd += coin_price*coin_amount
        return amount_usd
    
    def get_coin_market_cap(self, date, coin_symbol):
        return float(self.data[(self.data["Date"] == date) & (self.data["Symbol"] == coin_symbol)]["Market Cap"].values[0])
            
    def get_positions_range(self, date, first_position, last_position):
        coins_data = self.data[(self.data["Date"] == date) & (self.data["Pos"] <= last_position) & (self.data["Pos"] >= first_position)]
        for index, _ in coins_data["Symbol"].iteritems():
            yield coins_data["Symbol"][index]
    
    def dates(self):
        return sorted(self.data.Date.unique())
    
    def print_returns(self, initial_amount, initial_date, final_amount, final_date):
        print("%s: %s USD" % (initial_date, round(initial_amount, 2)))
        print("%s: %s USD" % (end_date, round(final_amount, 2)))
        print("----------")
        print("%sX" % round(final_amount/initial_amount, 1))
        

In [103]:
class InvestStrategies(object):
    def __init__(self, market):
        self.market = market

# Initialize

In [113]:
market = Coinmarketcap("data/coinmarketcap_formatted.tsv")
all_dates = market.dates()
weeks_ago = 52
initial_amount = 1000
start_date = all_dates[-weeks_ago]
end_date = all_dates[-1]

# Strategies

In [114]:
print("With an initial of %s USD from %s to %s" % (initial_amount, start_date, end_date))

With an initial of 1000 USD from 2016-05-01 to 2017-04-30


## Buy and forget

### Buy first 5 coins evenly

In [115]:
symbols_to_buy = market.get_positions_range(start_date, 1, 5)
coins = market.buy_coins(start_date, list(symbols_to_buy), initial_amount, strategy="even")
final_amount = market.sell_coins(end_date, coins)
market.print_returns(initial_amount, start_date, final_amount, end_date)

2016-05-01: 1000 USD
2017-04-30: 7353.04 USD
----------
7.4X


### Buy 50% Bitcoin 50% Top(2-5)

In [116]:
symbols_to_buy = list(market.get_positions_range(start_date, 1, 1))
coins = market.buy_coins(start_date, symbols_to_buy, initial_amount/2, strategy="even")
symbols_to_buy = list(market.get_positions_range(start_date, 2, 5))
top_coins = market.buy_coins(start_date, symbols_to_buy, initial_amount/2, strategy="even")
coins = {**coins, **top_coins}
final_amount = market.sell_coins(end_date, coins)
market.print_returns(initial_amount, start_date, final_amount, end_date)

2016-05-01: 1000 USD
2017-04-30: 5694.91 USD
----------
5.7X


### Buy Top 5 by market cap

In [117]:
symbols_to_buy = market.get_positions_range(start_date, 1, 5)
coins = market.buy_coins(start_date, list(symbols_to_buy), initial_amount, strategy="market_cap")
final_amount = market.sell_coins(end_date, coins)
market.print_returns(initial_amount, start_date, final_amount, end_date)

2016-05-01: 1000 USD
2017-04-30: 3582.94 USD
----------
3.6X


## Balancing every X week

### Buy first 5 coins evenly

In [118]:
coins = {}
for x_week in [1,2,3,4,8,12]:
    amount_to_buy = initial_amount
    for date in all_dates[-weeks_ago::x_week]:
        if date != start_date:
            amount_to_buy = market.sell_coins(date, coins)
        symbols_to_buy = market.get_positions_range(date, 1, 5)
        coins = market.buy_coins(date, list(symbols_to_buy), amount_to_buy, strategy="even")
    print("Balancing every %s weeks\n" % x_week)
    market.print_returns(initial_amount, start_date, amount_to_buy, end_date)

Balancing every 1 weeks

2016-05-01: 1000 USD
2017-04-30: 5226.62 USD
----------
5.2X
Balancing every 2 weeks

2016-05-01: 1000 USD
2017-04-30: 4073.69 USD
----------
4.1X
Balancing every 3 weeks

2016-05-01: 1000 USD
2017-04-30: 5916.11 USD
----------
5.9X
Balancing every 4 weeks

2016-05-01: 1000 USD
2017-04-30: 3284.55 USD
----------
3.3X
Balancing every 8 weeks

2016-05-01: 1000 USD
2017-04-30: 4111.26 USD
----------
4.1X
Balancing every 12 weeks

2016-05-01: 1000 USD
2017-04-30: 3420.77 USD
----------
3.4X


### Buy 50% Bitcoin 50% Top(2-5)

In [119]:
coins = {}
for x_week in [1,2,3,4,8,12]:
    for date in all_dates[-weeks_ago::x_week]:
        amount_to_buy = initial_amount
        if date != start_date:
            amount_to_buy = market.sell_coins(date, coins)
        symbols_to_buy = list(market.get_positions_range(start_date, 1, 1))
        coins = market.buy_coins(date, symbols_to_buy, amount_to_buy/2, strategy="even")
        symbols_to_buy = list(market.get_positions_range(start_date, 2, 5))
        top_coins = market.buy_coins(date, symbols_to_buy, amount_to_buy/2, strategy="even")
        coins = {**coins, **top_coins}
    print("Balancing every %s weeks\n" % x_week)
    market.print_returns(initial_amount, start_date, amount_to_buy, end_date)

Balancing every 1 weeks

2016-05-01: 1000 USD
2017-04-30: 6286.38 USD
----------
6.3X
Balancing every 2 weeks

2016-05-01: 1000 USD
2017-04-30: 5633.45 USD
----------
5.6X
Balancing every 3 weeks

2016-05-01: 1000 USD
2017-04-30: 8144.44 USD
----------
8.1X
Balancing every 4 weeks

2016-05-01: 1000 USD
2017-04-30: 5597.43 USD
----------
5.6X
Balancing every 8 weeks

2016-05-01: 1000 USD
2017-04-30: 4914.27 USD
----------
4.9X
Balancing every 12 weeks

2016-05-01: 1000 USD
2017-04-30: 4505.82 USD
----------
4.5X


### Buy Top 5 by market cap

In [120]:
coins = {}
for x_week in [1,2,3,4,8,12]:
    for date in all_dates[-weeks_ago::x_week]:
        amount_to_buy = initial_amount
        if date != start_date:
            amount_to_buy = market.sell_coins(date, coins)
        symbols_to_buy = market.get_positions_range(start_date, 1, 5)
        coins = market.buy_coins(date, list(symbols_to_buy), amount_to_buy, strategy="market_cap")
        amount_to_buy = market.sell_coins(date, coins)
    print("Balancing every %s weeks\n" % x_week)
    market.print_returns(initial_amount, start_date, amount_to_buy, end_date)

Balancing every 1 weeks

2016-05-01: 1000 USD
2017-04-30: 3625.58 USD
----------
3.6X
Balancing every 2 weeks

2016-05-01: 1000 USD
2017-04-30: 3107.88 USD
----------
3.1X
Balancing every 3 weeks

2016-05-01: 1000 USD
2017-04-30: 3623.54 USD
----------
3.6X
Balancing every 4 weeks

2016-05-01: 1000 USD
2017-04-30: 2955.49 USD
----------
3.0X
Balancing every 8 weeks

2016-05-01: 1000 USD
2017-04-30: 2954.96 USD
----------
3.0X
Balancing every 12 weeks

2016-05-01: 1000 USD
2017-04-30: 2951.77 USD
----------
3.0X


## Buy 50% bitcoin 25% Top2-5 marketcap 25% top2-10 evenly

In [121]:
coins = {}
for x_week in [1,2,3,4,8,12]:
    for date in all_dates[-weeks_ago::x_week]:
        amount_to_buy = initial_amount
        if date != start_date:
            amount_to_buy = market.sell_coins(date, coins)
        symbols_to_buy = list(market.get_positions_range(date, 1, 1))
        coins = market.buy_coins(date, symbols_to_buy, amount_to_buy/2, strategy="even")
        
        symbols_to_buy = list(market.get_positions_range(date, 2, 5))
        top_2_5_coins = market.buy_coins(date, symbols_to_buy, (amount_to_buy/2)/2, strategy="market_cap")
        
        symbols_to_buy = list(market.get_positions_range(date, 2, 10))
        top_2_10_coins = market.buy_coins(date, symbols_to_buy, (amount_to_buy/2)/2, strategy="even")
        
        coins_keys = set(list(coins.keys()) + list(top_2_5_coins.keys()) + list(top_2_10_coins.keys()))
        new_coins = {}
        for coin_key in coins_keys:
            new_coins[coin_key] = coins.get(coin_key, 0) + top_2_5_coins.get(coin_key, 0) + top_2_10_coins.get(coin_key, 0)
        coins = new_coins
    print("Balancing every %s weeks\n" % x_week)
    market.print_returns(initial_amount, start_date, amount_to_buy, end_date)

Balancing every 1 weeks

2016-05-01: 1000 USD
2017-04-30: 5086.73 USD
----------
5.1X
Balancing every 2 weeks

2016-05-01: 1000 USD
2017-04-30: 4085.51 USD
----------
4.1X
Balancing every 3 weeks

2016-05-01: 1000 USD
2017-04-30: 5372.14 USD
----------
5.4X
Balancing every 4 weeks

2016-05-01: 1000 USD
2017-04-30: 3364.66 USD
----------
3.4X
Balancing every 8 weeks

2016-05-01: 1000 USD
2017-04-30: 4192.79 USD
----------
4.2X
Balancing every 12 weeks

2016-05-01: 1000 USD
2017-04-30: 3787.51 USD
----------
3.8X
